In [1]:
import torch
from torch.nn import functional as F

import pandas as pd
import swifter
from tqdm.notebook import tqdm
import csv
import pickle
import time

/usr/local/lib/python3.7/dist-packages/numba/errors.py:137: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


In [2]:
data_path = '../../data_2020/'
model_path = 'models/'
other_path = 'others/'
prefix = 'test'
postpostfix = '_'+prefix
MIN_LEN = 7
t1 = time.time()
paper_thd = 45000

In [3]:
with open(other_path+'description2embedding_blue.pkl', 'rb') as f:
    description2embedding = pickle.load(f)
with open(other_path+'paper2embedding_blue.pkl', 'rb') as f:
    paper2embedding = pickle.load(f)
    
data = pd.read_csv(data_path+prefix+'_pre.csv')[['description_id', 'key_text_pre']].fillna('none')
paper = pd.read_csv(data_path+'candidate_paper_pre.csv')['paper_id'].values
dids = ['' if did == 'none' else did for did in data['description_id'].values]

pid2idx = {}
idx2pid = {}
for i, pid in enumerate(tqdm(paper)):
    pid2idx[pid] = len(pid2idx)
    idx2pid[len(pid2idx)-1] = pid
assert len(pid2idx) == len(paper)

data['key_text_len'] = data['key_text_pre'].apply(lambda x: len(x.split()))
data = data.values

data_embs = [description2embedding[row[0]+postpostfix]['description_text'] if row[2] < MIN_LEN \
             else description2embedding[row[0]+postpostfix]['key_text'] for row in data]
paper_embs = [paper2embedding[pid]['title'] for pid in paper]

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
print('generating tensors...')
data_embs = torch.FloatTensor(data_embs)
paper_embs = torch.FloatTensor(paper_embs)
data_embs = F.normalize(data_embs, p=2, dim=1)
paper_embs = F.normalize(paper_embs, p=2, dim=1)

batch_size = 2048
steps = data_embs.size(0)//batch_size + 1
topis = []

for step in tqdm(range(steps)):
    data_embs_b = data_embs[step*batch_size: (step+1)*batch_size]
    results = torch.mm(data_embs_b, paper_embs.transpose(0, 1))
    _, topi = torch.topk(results, 1000)
    topis += topi.tolist()

with open(data_path+prefix+'_pairs_1000number_bluebert_2.csv', 'w', newline='') as f:
    w = csv.writer(f)
    for i in range(data.shape[0]):
        w.writerow([dids[i]]+[idx2pid[idx] for idx in topis[i]])
        
t2 = time.time()
t2-t1

generating tensors...


84.02835202217102

### recall score for train
***

In [5]:
data = pd.read_csv(data_path+prefix+'_pre.csv')[['description_id', 'paper_id']].values
hit = 0

for i in tqdm(range(data.shape[0])):
    for j in range(3):
        if topis[i][j] == pid2idx[data[i][1]]:
            hit += 1
            break
            
hit/data.shape[0]

KeyError: "['paper_id'] not in index"